In [14]:
from bs4 import BeautifulSoup
import pymysql
import requests
# import library

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('darkgrid')
sns.set_palette('Set3')

import missingno as msno
import warnings
warnings.filterwarnings('ignore')

import scipy.stats

plt.rcParams["font.family"] = "NanumBarunGothic"

from category_encoders.target_encoder import TargetEncoder

import re

In [15]:
# data load
# pip install openpyxl
path = "C:"
# Top_Lib = pd.read_excel(path + '/크롤링select데이터/나머지.xlsx')
Top_Lib = pd.read_csv(path + '/all_data.csv')


# ISBN_THIRTEEN_NO 컬럼의 문자열이 13글자 이상인 행만 선택
Top_Lib_group = Top_Lib[Top_Lib['ISBN_THIRTEEN_NO'].str.len() >= 13]

In [16]:
# 데이터를 4등분하여 각각의 데이터프레임으로 저장
num_rows = len(Top_Lib_group)
chunk_size = num_rows // 6

# 4개의 데이터프레임을 저장할 리스트 생성
data_chunks = [Top_Lib_group.iloc[i*chunk_size:(i+1)*chunk_size] for i in range(6)]

# 각각의 데이터프레임을 다른 변수에 할당
df1, df2, df3, df4, df5, df6 = data_chunks

# 결과 출력 (예시로 첫 번째 데이터프레임 출력)
print("첫 번째 데이터프레임:")
print(df4)


num_rows, num_columns = df4.shape

# 결과 출력
print(f"데이터 수: {num_rows}")

plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

첫 번째 데이터프레임:
            SEQ_NO  RANK_CO     INPT_DE ISBN_THIRTEEN_NO  \
2113536  128617572      389  2019-02-15    9788960861510   
2113537  128617573      389  2019-02-15    9788960864016   
2113538  128617571      389  2019-02-15    9788960861268   
2113539  128617570      389  2019-02-15    9788960861138   
2113540  128617560      389  2019-02-15    9788959762323   
...            ...      ...         ...              ...   
3320486  188988908      798  2020-01-15    9788985804394   
3320487  188988909      798  2020-01-15    9788993964141   
3320488  188988910      798  2020-01-15    9788994120997   
3320489  188988865      798  2020-01-15    9788933801888   
3320490  188988866      798  2020-01-15    9788934999232   

                                             BOOK_TITLE_NM         AUTHR_NM  \
2113536                                                  꼴         허영만 글·그림   
2113537  30일 완성 목소리 트레이닝 =아나운서 우지은의 호감형 목소리 만들기 /Voice ...           우지은 지음   
2113538                      

In [17]:
import csv
import os

conn = pymysql.connect(host='127.0.0.1',
                       user='root',
                       password='1234',
                       db='Test_one',
                       charset='utf8')
cursor = conn.cursor()
headers = {
   'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

Book_count = 0

for index, count in df4.iterrows():
    response = requests.get("https://search.kyobobook.co.kr/search?keyword={0}&gbCode=TOT&target=total".format(count['ISBN_THIRTEEN_NO']), headers=headers)
        
    html = response.content.decode('utf-8','replace')
    soup = BeautifulSoup(html, 'html.parser',from_encoding='cp949')

    my_st = soup.select(
        '#shopData_list > ul > li > div.prod_area.horizontal > div.prod_info_box'
    )
    
    for stock_name in my_st:
        try:
            price_elements = stock_name.select("#shopData_list > ul > li > div.prod_area.horizontal > div.prod_info_box > div.prod_price > span.price > span.val")

            print(price_elements)
            if price_elements == []:
                Price_varchar = -1
            else:
                if price_elements and price_elements[0]:
                    Book_Price = price_elements[0].text
                    Price_varchar = Book_Price.replace(',','')
                    # print(Price_varchar)
                    
                else:
                    Book_Price = "0"

        except IndexError as e:
            print("Book_Price 처리 중 에러:", e)
            Book_Price = "0"
        
        try:
            Vente_exposant = stock_name.select("#shopData_list > ul > li > div.prod_area.horizontal > div.prod_info_box > div.prod_bottom > a > span.review_klover_box")
        
            if Vente_exposant and Vente_exposant[0]:
                Vente_exposant_text = Vente_exposant[0].text

                Vente_exposant_text_one = '%-11s' % (' ').join(Vente_exposant_text.split())
                matches = re.findall(r'[\d.]+', Vente_exposant_text_one)
                # print(Vente_exposant_text_one)
                ranking = matches[0]
            else:
                Vente_exposant_text = "0"

        except IndexError as e:
            print("review_desc 처리 중 에러:", e)
            Vente_exposant_text = "0"

        try:
            Vente_exposant = stock_name.select("#shopData_list > ul > li > div.prod_area.horizontal > div.prod_info_box > div.prod_bottom > a > span.review_klover_box")
        
            if Vente_exposant and Vente_exposant[0]:
                Vente_exposant_text = Vente_exposant[0].text

                Vente_exposant_text_one = '%-11s' % (' ').join(Vente_exposant_text.split())
                matches = re.findall(r'[\d.]+', Vente_exposant_text_one)
                # print(Vente_exposant_text_one)
                review_desc = matches[1]
            else:
                Vente_exposant_text = "0"

        except IndexError as e:
            print("review_desc 처리 중 에러:", e)
            Vente_exposant_text = "0"
       
    
    Book_count += 1
        # Replace '무료도서' with 0 for the 'price' column
    ISBN = str(count['ISBN_THIRTEEN_NO'])
    PRICE = 0 if Price_varchar == '무료도서' or Price_varchar == '절판되었습니다.' else int(Price_varchar) if Price_varchar is not None else 0
    Ranking = ranking
    Review_C =  review_desc

    # CSV 파일 경로
    csv_file_path = 'C:/프로젝트/final_Kyobo4.csv'

    # 데이터를 리스트로 만들기
    data = [[str(ISBN), str(PRICE), str(Ranking), str(Review_C)]]

    # 첫 번째 데이터를 쓸 때만 헤더를 작성하고, 이후에는 헤더를 작성하지 않음
    write_header = True

    # CSV 파일에 데이터 쓰기 (기존 파일에 이어서 쓰기 모드인 'a'를 사용)
    with open(csv_file_path, 'a', newline='', encoding='utf-8') as csvfile:
        # CSV writer 생성
        csv_writer = csv.writer(csvfile)

        # 헤더 작성
        
        csv_writer.writerow(data[0])

    print(ISBN, PRICE,Ranking,Review_C)
    print(Book_count, "/", {num_rows})

[<span class="val">13,500</span>]
9788960861510 13500 9.53 11
1 / {27306}
[]
9788960864016 -1 10.0 3
2 / {27306}
[<span class="val">13,500</span>]
9788960861268 13500 9.12 12
3 / {27306}
[<span class="val">13,500</span>]
9788960861138 13500 8.21 32
4 / {27306}
[<span class="val">10,800</span>]
9788959762323 10800 9.48 74
5 / {27306}
[<span class="val">7,200</span>]
9788959777402 7200 9.56 56
6 / {27306}
[]
9788960243125 -1 10.0 3
7 / {27306}
[]
9788960243163 -1 0.0 0
8 / {27306}
[]
9788960243187 -1 9.12 10
9 / {27306}
[<span class="val">8,550</span>]
9788960243392 8550 0.0 0
10 / {27306}
[<span class="val">8,550</span>]
9788960243453 8550 10.0 2
11 / {27306}
[<span class="val">16,200</span>]
9788960516175 16200 9.38 516
12 / {27306}
[]
9788960532410 -1 9.75 285
13 / {27306}
[]
9788960532427 -1 9.71 96
14 / {27306}
9788959759552 -1 9.71 96
15 / {27306}
[<span class="val">11,520</span>]
9788959759590 11520 9.29 35
16 / {27306}
[<span class="val">10,800</span>]
9788959761906 10800 9.33 61